In [ ]:
import heapq
import numpy as np

class Puzzle:
    def __init__(self, board, parent=None, move=None, depth=0, cost=0):
        self.board = np.array(board)
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.zero_pos = tuple(map(int, np.where(self.board == 0)))

    def __lt__(self, other):
        return (self.cost + self.heuristic()) < (other.cost + other.heuristic())

    def heuristic(self):
        goal = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
        return sum(abs(x - gx) + abs(y - gy) for x, y in np.ndindex(self.board.shape) if self.board[x, y] and (gx := np.where(goal == self.board[x, y])[0][0], gy := np.where(goal == self.board[x, y])[1][0]))

    def get_neighbors(self):
        x, y = self.zero_pos
        neighbors = []
        moves = {'Up': (x - 1, y), 'Down': (x + 1, y), 'Left': (x, y - 1), 'Right': (x, y + 1)}
        for move, (nx, ny) in moves.items():
            if 0 <= nx < 3 and 0 <= ny < 3:
                new_board = self.board.copy()
                new_board[x, y], new_board[nx, ny] = new_board[nx, ny], new_board[x, y]
                neighbors.append(Puzzle(new_board, self, move, self.depth + 1, self.cost + 1))
        return neighbors

def solve_puzzle(start):
    start_puzzle = Puzzle(start)
    pq = []
    heapq.heappush(pq, start_puzzle)
    visited = set()

    while pq:
        current = heapq.heappop(pq)
        if np.array_equal(current.board, np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])):
            path = []
            while current.parent:
                path.append(current.move)
                current = current.parent
            return path[::-1]

        visited.add(tuple(map(tuple, current.board)))
        for neighbor in current.get_neighbors():
            if tuple(map(tuple, neighbor.board)) not in visited:
                heapq.heappush(pq, neighbor)
    return None

# Trạng thái ban đầu từ hình
initial_state = [[5, 4, 0], [6, 1, 8], [7, 3, 2]]
solution = solve_puzzle(initial_state)
print("Solution steps:", solution)


<ipython-input-1-c303e4fdec68>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.zero_pos = tuple(map(int, np.where(self.board == 0)))


Solution steps: ['Down', 'Left', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down']


In [ ]:
from queue import Queue
import copy

# Định nghĩa trạng thái đích (Goal State)
goal_state = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# Kiểm tra xem trạng thái hiện tại có phải là trạng thái đích không
def is_goal(state):
    return state == goal_state

# Tìm vị trí ô trống (0) trong ma trận
def find_empty(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j
    return -1, -1

# Sinh ra các trạng thái kế tiếp từ trạng thái hiện tại
def get_neighbors(state):
    x, y = find_empty(state)
    neighbors = []
    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Lên, Xuống, Trái, Phải
    for dx, dy in moves:
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = copy.deepcopy(state)
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            neighbors.append(new_state)
    return neighbors

# Tìm đường đi bằng BFS
def bfs(start_state):
    queue = Queue()
    queue.put((start_state, []))
    visited = set()
    visited.add(str(start_state))
    while not queue.empty():
        current_state, path = queue.get()
        if is_goal(current_state):
            return path
        for neighbor in get_neighbors(current_state):
            if str(neighbor) not in visited:
                visited.add(str(neighbor))
                queue.put((neighbor, path + [neighbor]))
    return None

# Trạng thái bắt đầu
start_state = [[7, 2, 4], [5, 0, 6], [8, 3, 1]]

# Giải bài toán
solution = bfs(start_state)

if solution:
    print("Đã tìm thấy giải pháp:")
    for step in solution:
        for row in step:
            print(row)
        print("---")
else:
    print("Không tìm thấy giải pháp")


Đã tìm thấy giải pháp:
[7, 2, 4]
[5, 3, 6]
[8, 0, 1]
---
[7, 2, 4]
[5, 3, 6]
[8, 1, 0]
---
[7, 2, 4]
[5, 3, 0]
[8, 1, 6]
---
[7, 2, 4]
[5, 0, 3]
[8, 1, 6]
---
[7, 2, 4]
[0, 5, 3]
[8, 1, 6]
---
[0, 2, 4]
[7, 5, 3]
[8, 1, 6]
---
[2, 0, 4]
[7, 5, 3]
[8, 1, 6]
---
[2, 4, 0]
[7, 5, 3]
[8, 1, 6]
---
[2, 4, 3]
[7, 5, 0]
[8, 1, 6]
---
[2, 4, 3]
[7, 0, 5]
[8, 1, 6]
---
[2, 4, 3]
[7, 1, 5]
[8, 0, 6]
---
[2, 4, 3]
[7, 1, 5]
[0, 8, 6]
---
[2, 4, 3]
[0, 1, 5]
[7, 8, 6]
---
[2, 4, 3]
[1, 0, 5]
[7, 8, 6]
---
[2, 0, 3]
[1, 4, 5]
[7, 8, 6]
---
[0, 2, 3]
[1, 4, 5]
[7, 8, 6]
---
[1, 2, 3]
[0, 4, 5]
[7, 8, 6]
---
[1, 2, 3]
[4, 0, 5]
[7, 8, 6]
---
[1, 2, 3]
[4, 5, 0]
[7, 8, 6]
---
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]
---


In [8]:
import copy

# Kiểm tra trạng thái thắng
def check_win(board, player):
    win_states = [
        [(0, 0), (0, 1), (0, 2)],  # Hàng ngang
        [(1, 0), (1, 1), (1, 2)],
        [(2, 0), (2, 1), (2, 2)],
        [(0, 0), (1, 0), (2, 0)],  # Hàng dọc
        [(0, 1), (1, 1), (2, 1)],
        [(0, 2), (1, 2), (2, 2)],
        [(0, 0), (1, 1), (2, 2)],  # Chéo
        [(0, 2), (1, 1), (2, 0)]
    ]
    for line in win_states:
        if all(board[x][y] == player for x, y in line):
            return True
    return False

# In bàn cờ
def print_board(board):
    for row in board:
        print(" ".join(row))
    print("---")

# Lấy các nước đi hợp lệ
def get_moves(board):
    moves = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == '.':
                moves.append((i, j))
    return moves

# Tính điểm cho Minimax
def evaluate(board):
    if check_win(board, 'X'):
        return 1
    elif check_win(board, 'O'):
        return -1
    return 0

# Thuật toán Minimax có cắt tỉa alpha-beta
def minimax(board, depth, is_max):
    score = evaluate(board)
    if score == 1 or score == -1:
        return score
    if not get_moves(board):
        return 0

    if is_max:
        best = -float('inf')
        for move in get_moves(board):
            board[move[0]][move[1]] = 'X'
            best = max(best, minimax(board, depth + 1, False))
            board[move[0]][move[1]] = '.'
        return best
    else:
        best = float('inf')
        for move in get_moves(board):
            board[move[0]][move[1]] = 'O'
            best = min(best, minimax(board, depth + 1, True))
            board[move[0]][move[1]] = '.'
        return best

# Tìm nước đi tối ưu cho X
def find_best_move(board):
    best_val = -float('inf')
    best_move = None
    for move in get_moves(board):
        board[move[0]][move[1]] = 'X'
        move_val = minimax(board, 0, False)
        board[move[0]][move[1]] = '.'
        if move_val > best_val:
            best_val = move_val
            best_move = move
    return best_move

# Bàn cờ ban đầu
board = [['.', '.', '.'], ['.', '.', '.'], ['.', '.', '.']]
print("Bạn là O. Chọn vị trí bằng cách nhập hàng và cột (0-2).")
print_board(board)

while True:
    # Người chơi O chọn vị trí
    row, col = map(int, input("Nhập hàng và cột (cách nhau bằng dấu cách): ").split())
    if board[row][col] == '.':
        board[row][col] = 'O'
    else:
        print("Vị trí không hợp lệ!")
        continue
    print("Bàn cờ sau khi O đi:")
    print_board(board)
    if check_win(board, 'O'):
        print("Người chơi O thắng!")
        break
    if not get_moves(board):
        print("Hòa!")
        break

    # Máy đi X
    move = find_best_move(board)
    if move:
        board[move[0]][move[1]] = 'X'
        print("Bàn cờ sau khi X đi:")
        print_board(board)
        if check_win(board, 'X'):
            print("Máy (X) thắng!")
            break
    else:
        print("Hòa!")
        break


Bạn là O. Chọn vị trí bằng cách nhập hàng và cột (0-2).
. . .
. . .
. . .
---
Nhập hàng và cột (cách nhau bằng dấu cách): 1 1
Bàn cờ sau khi O đi:
. . .
. O .
. . .
---
Bàn cờ sau khi X đi:
X . .
. O .
. . .
---
Nhập hàng và cột (cách nhau bằng dấu cách): 2 2
Bàn cờ sau khi O đi:
X . .
. O .
. . O
---
Bàn cờ sau khi X đi:
X . X
. O .
. . O
---
Nhập hàng và cột (cách nhau bằng dấu cách): 0 2
Vị trí không hợp lệ!
Nhập hàng và cột (cách nhau bằng dấu cách): 2 0
Bàn cờ sau khi O đi:
X . X
. O .
O . O
---
Bàn cờ sau khi X đi:
X X X
. O .
O . O
---
Máy (X) thắng!


In [9]:
from collections import deque

# Hàm BFS
def bfs(graph, start, goal):
    fringe = deque([start])  # Hàng đợi fringe (FIFO)
    closed = set()          # Tập các nút đã duyệt

    while fringe:
        node = fringe.popleft()  # Lấy phần tử đầu tiên
        closed.add(node)
        print(f"Đang duyệt: {node}")

        # Kiểm tra nếu đã tìm thấy đích
        if node == goal:
            print("Đã tìm thấy đích!", node)
            return True

        # Duyệt các trạng thái con
        for neighbor in graph.get(node, []):
            if neighbor not in closed and neighbor not in fringe:
                fringe.append(neighbor)
                print(f"Thêm vào fringe: {neighbor}")

    print("Không tìm thấy giải pháp.")
    return False

# Ví dụ đồ thị biểu diễn bằng danh sách kề
graph = {
    'A': ['B', 'C', 'D'],
    'B': ['E', 'F'],
    'C': ['G'],
    'D': ['H'],
    'E': [],
    'F': [],
    'G': [],
    'H': []
}

# Gọi hàm BFS
start_node = 'A'
goal_node = 'G'
bfs(graph, start_node, goal_node)


Đang duyệt: A
Thêm vào fringe: B
Thêm vào fringe: C
Thêm vào fringe: D
Đang duyệt: B
Thêm vào fringe: E
Thêm vào fringe: F
Đang duyệt: C
Thêm vào fringe: G
Đang duyệt: D
Thêm vào fringe: H
Đang duyệt: E
Đang duyệt: F
Đang duyệt: G
Đã tìm thấy đích! G


True

In [10]:
import heapq

def ucs(graph, start, goal):
    fringe = []
    heapq.heappush(fringe, (0, start))
    closed = set()

    while fringe:
        cost, node = heapq.heappop(fringe)
        closed.add(node)
        print(f"Đang duyệt: {node} với chi phí: {cost}")

        if node == goal:
            print("Đã tìm thấy đích!", node)
            return True

        for neighbor, weight in graph.get(node, []):
            if neighbor not in closed:
                new_cost = cost + weight
                heapq.heappush(fringe, (new_cost, neighbor))
                print(f"Thêm vào fringe: {neighbor} với chi phí: {new_cost}")

    print("Không tìm thấy giải pháp.")
    return False

graph = {
    'A': [('B', 1), ('C', 4), ('D', 3)],
    'B': [('E', 5), ('F', 1)],
    'C': [('G', 2)],
    'D': [('H', 7)],
    'E': [],
    'F': [],
    'G': [],
    'H': []
}

start_node = 'A'
goal_node = 'G'
ucs(graph, start_node, goal_node)


Đang duyệt: A với chi phí: 0
Thêm vào fringe: B với chi phí: 1
Thêm vào fringe: C với chi phí: 4
Thêm vào fringe: D với chi phí: 3
Đang duyệt: B với chi phí: 1
Thêm vào fringe: E với chi phí: 6
Thêm vào fringe: F với chi phí: 2
Đang duyệt: F với chi phí: 2
Đang duyệt: D với chi phí: 3
Thêm vào fringe: H với chi phí: 10
Đang duyệt: C với chi phí: 4
Thêm vào fringe: G với chi phí: 6
Đang duyệt: E với chi phí: 6
Đang duyệt: G với chi phí: 6
Đã tìm thấy đích! G


True

In [16]:
def dfs(graph, start, goal):
    fringe = [start]
    closed = set()
    parent = {start: None}

    while fringe:
        n = fringe.pop()

        closed.add(n)

        if n == goal:
            path = []
            while n is not None:
                path.append(n)
                n = parent[n]
            path.reverse()
            return f"Đường đi: {' -> '.join(path)}"

        if n in graph:
            for neighbor in graph[n]:
                if neighbor not in closed:
                    fringe.append(neighbor)
                    parent[neighbor] = n

    return "Không tìm thấy đường đi"

graph = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F'],
    'D': [],
    'E': ['F'],
    'F': ['G'],
    'G': []
}

start_node = 'A'
goal_node = 'G'
result = dfs(graph, start_node, goal_node)
print(result)


Đường đi: A -> C -> F -> G


In [19]:
def depth_limited_search(problem, limit):
    initial_node = make_node(problem.initial_state, 0)
    return recursive_dls(initial_node, problem, limit)

def recursive_dls(node, problem, limit):
    if node.depth > limit:
        return "cutoff"

    if goal_test(problem, node.state):
        return solution(node)

    cutoff_occurred = False

    for successor in expand(node, problem):
        result = recursive_dls(successor, problem, limit)

        if result == "cutoff":
            cutoff_occurred = True
        elif result != "failure":
            return result

    if cutoff_occurred:
        return "cutoff"
    return "failure"

def make_node(state, depth):
    return Node(state, depth)

def goal_test(problem, state):
    return state == problem.goal

def expand(node, problem):
    successors = []
    for action in problem.actions(node.state):
        new_state = problem.result(node.state, action)
        successors.append(make_node(new_state, node.depth + 1))
    return successors

def solution(node):
    return f"Solution found at state: {node.state}"

class Node:
    def __init__(self, state, depth):
        self.state = state
        self.depth = depth

# Ví dụ về bài toán:
class Problem:
    def __init__(self, initial_state, goal):
        self.initial_state = initial_state
        self.goal = goal

    def actions(self, state):
        return ['action1', 'action2']

    def result(self, state, action):
        return state

problem = Problem(initial_state="A", goal="G")
limit = 3
result = depth_limited_search(problem, limit)
print(result)


cutoff


In [27]:
from collections import deque

class Node:
    def __init__(self, state, parent=None, move=None, depth=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth

    def __str__(self):
        return str(self.state)

    def path(self):
        path = []
        node = self
        while node is not None:
            path.append(node)
            node = node.parent
        path.reverse()
        return path


def depth_limited_search(problem, limit):
    initial_node = make_node(problem.initial_state, 0)
    return recursive_dls(initial_node, problem, limit)


def recursive_dls(node, problem, limit):
    if node.depth > limit:
        return "cutoff"

    if goal_test(problem, node.state):
        return node

    cutoff_occurred = False

    for successor in expand(node, problem):
        result = recursive_dls(successor, problem, limit)

        if result == "cutoff":
            cutoff_occurred = True
        elif result != "failure":
            return result

    if cutoff_occurred:
        return "cutoff"

    return "failure"


def make_node(state, depth):
    return Node(state, depth=depth)


def goal_test(problem, state):
    return state == problem.goal


def expand(node, problem):
    successors = []
    zero_pos = node.state.index(0)
    row, col = divmod(zero_pos, 3)

    possible_moves = [
        (-1, 0), (1, 0), (0, -1), (0, 1)
    ]

    for move in possible_moves:
        new_row, new_col = row + move[0], col + move[1]
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_zero_pos = new_row * 3 + new_col
            new_state = list(node.state)

            new_state[zero_pos], new_state[new_zero_pos] = new_state[new_zero_pos], new_state[zero_pos]
            successors.append(Node(new_state, parent=node, move=(row, col, new_row, new_col), depth=node.depth + 1))

    return successors


class Problem:
    def __init__(self, initial_state, goal):
        self.initial_state = initial_state
        self.goal = goal


initial_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]

problem = Problem(initial_state=initial_state, goal=goal_state)

limit = 10

result = depth_limited_search(problem, limit)

if result != "failure" and result != "cutoff":
    print("Đường đi giải quyết bài toán:")
    for node in result.path():
        print(node.state)
else:
    print(result)


Đường đi giải quyết bài toán:
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [25]:
def depth_limited_search(problem, depth_limit):
    def recursive_dls(node, depth):
        if problem.goal_test(node):
            return node
        elif depth == 0:
            return "cutoff"
        else:
            cutoff_occurred = False
            for child in problem.expand(node):
                result = recursive_dls(child, depth - 1)
                if result == "cutoff":
                    cutoff_occurred = True
                elif result != "failure":
                    return result
            return "cutoff" if cutoff_occurred else "failure"

    return recursive_dls(problem.initial_state, depth_limit)

def iterative_deepening_search(problem):
    depth = 0
    while True:
        result = depth_limited_search(problem, depth)
        if result != "cutoff":
            return result
        depth += 1

class SimpleProblem:
    def __init__(self, initial_state, goal_state):
        self.initial_state = initial_state
        self.goal_state = goal_state

    def goal_test(self, state):
        return state == self.goal_state

    def expand(self, state):
        return [state + 1, state - 1]

problem = SimpleProblem(0, 5)

result = iterative_deepening_search(problem)
print(f"Result: {result}")

Result: 5
